# Build a Multi-User Conversational Tool-Calling Agentic AI Research Assistant with LangChain

This demo will cover building AI Agents with the legacy LangChain `AgentExecutor`. These are fine for getting started, but for working with more advanced agents, LangChain recommends to use LangGraph, which we will cover in the future demos.

Agents are systems that use an LLM as a reasoning engine to determine which actions to take and what the inputs to those actions should be. The results of those actions can then be fed back into the agent and it determines whether more actions are needed, or whether it is okay to stop.

Here we will build a multi-user Conversational Agent which can refer to previous conversations and give more contextual answers by storing agent messages to a SQL database

![](https://i.imgur.com/lHWqaT9.png)



## Install OpenAI, and LangChain dependencies

In [1]:
!pip install -qq langchain==0.3.14
!pip install -qq langchain-openai==0.3.0
!pip install -qq langchain-community==0.3.14

In [2]:
!pip install -qq markitdown

## Enter Open AI API Key

In [ ]:
# from getpass import getpass

# OPENAI_KEY = getpass('Enter Open AI API Key: ')

Enter Open AI API Key: ··········


## Enter Tavily Search API Key

Get a free API key from [here](https://tavily.com/#api)

In [ ]:
# TAVILY_API_KEY = getpass('Enter Tavily Search API Key: ')

Enter Tavily Search API Key: ··········


## Enter WeatherAPI API Key

Get a free API key from [here](https://www.weatherapi.com/signup.aspx)

In [ ]:
# WEATHER_API_KEY = getpass('Enter WeatherAPI API Key: ')

Enter WeatherAPI API Key: ··········


In [4]:
import os
from dotenv import load_dotenv
load_dotenv()

True

## Setup Environment Variables

In [ ]:
# import os

# os.environ['OPENAI_API_KEY'] = OPENAI_KEY
# os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY

## Create Tools

Here we create two custom tools which are wrappers on top of the [Tavily API](https://tavily.com/#api) and [WeatherAPI](https://www.weatherapi.com/)

- Web Search tool with information extraction
- Weather tool

![](https://i.imgur.com/TyPAYXE.png)

In [7]:
WEATHER_API_KEY = os.environ['OPENAI_API_KEY']

In [13]:
from langchain_core.tools import tool
from markitdown import MarkItDown
from langchain_community.tools.tavily_search import TavilySearchResults
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, TimeoutError
import requests
import json
from warnings import filterwarnings
filterwarnings('ignore')

tavily_tool = TavilySearchResults(max_results=5,
                                  search_depth='advanced',
                                  include_answer=False,
                                  include_raw_content=True)
session = requests.Session()
session.headers.update({
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br"
})
md = MarkItDown(requests_session=session)

@tool
def search_web_extract_info(query: str) -> list:
    """Search the web for a query and extracts useful information from the search links."""
    print('Calling web search tool')
    results = tavily_tool.invoke(query)
    docs = []

    def extract_content(url):
        """Helper function to extract content from a URL."""
        extracted_info = md.convert(url)
        text_title = extracted_info.title.strip()
        text_content = extracted_info.text_content.strip()
        return text_title + '\n' + text_content

    with ThreadPoolExecutor() as executor:
        for result in tqdm(results):
            try:
                future = executor.submit(extract_content, result['url'])
                # Wait for up to 60 seconds for the task to complete
                content = future.result(timeout=60)
                docs.append(content)
            except TimeoutError:
                print(f"Extraction timed out for url: {result['url']}")
            except Exception as e:
                print(f"Error extracting from url: {result['url']} - {e}")

    return docs


@tool
def get_weather(query: str) -> list:
    """Search weatherapi to get the current weather of the queried location."""
    print('Calling weather tool')
    base_url = "http://api.weatherapi.com/v1/current.json"
    complete_url = f"{base_url}?key={WEATHER_API_KEY}&q={query}"

    response = requests.get(complete_url)
    data = response.json()
    if data.get("location"):
        return data
    else:
        return "Weather Data Not Found"

In [14]:
get_weather.invoke("weather in kolkata now")

Calling weather tool


'Weather Data Not Found'

## Build and Test AI Agent

Now that we have defined the tools and the LLM, we can create the agent. We will be using a tool calling agent to bind the tools to the agent with a prompt. We will also add in the capability to store historical conversations as memory

In [15]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

SYS_PROMPT = """Act as a helpful assistant.
                You run in a loop of Thought, Action, PAUSE, Observation.
                At the end of the loop, you output an Answer.
                Use Thought to describe your thoughts about the question you have been asked.
                Use Action to run one of the actions available to you - then return PAUSE.
                Observation will be the result of running those actions.
                Repeat till you get to the answer for the given user query.

                Use the following workflow format:
                  Question: the input task you must solve
                  Thought: you should always think about what to do
                  Action: the action to take which can be any of the following:
                            - break it into smaller steps if needed
                            - see if you can answer the given task with your trained knowledge
                            - call the most relevant tools at your disposal mentioned below in case you need more information
                  Action Input: the input to the action
                  Observation: the result of the action
                  ... (this Thought/Action/Action Input/Observation can repeat N times)
                  Thought: I now know the final answer
                  Final Answer: the final answer to the original input question

                Tools at your disposal to perform tasks as needed:
                  - get_weather: whenever user asks get the weather of a place.
                  - search_web_extract_info: whenever user asks for specific information or if you don't know the answer.
             """

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", SYS_PROMPT),
        MessagesPlaceholder(variable_name="history", optional=True),
        ("human", "{query}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

prompt_template.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="Act as a helpful assistant.\n                You run in a loop of Thought, Action, PAUSE, Observation.\n                At the end of the loop, you output an Answer.\n                Use Thought to describe your thoughts about the question you have been asked.\n                Use Action to run one of the actions available to you - then return PAUSE.\n                Observation will be the result of running those actions.\n                Repeat till you get to the answer for the given user query.\n\n                Use the following workflow format:\n                  Question: the input task you must solve\n                  Thought: you should always think about what to do\n                  Action: the action to take which can be any of the following:\n                            - break it into smaller steps if needed\n                            - see if you can

Now, we can initalize the agent with the LLM, the prompt, and the tools.

The agent is responsible for taking in input and deciding what actions to take.

REMEMBER the Agent does not execute those actions - that is done by the AgentExecutor

Note that we are passing in the model `chatgpt`, not `chatgpt_with_tools`.

That is because `create_tool_calling_agent` will call `.bind_tools` for us under the hood.

This should ideally be used with an LLM which supports tool \ function calling

In [16]:
from langchain.agents import create_tool_calling_agent
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model="gpt-4o", temperature=0)
tools = [search_web_extract_info, get_weather]
agent = create_tool_calling_agent(chatgpt, tools, prompt_template)

Finally, we combine the `agent` (the brains) with the `tools` inside the `AgentExecutor` (which will repeatedly call the agent and execute tools).

In [17]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent,
                               tools=tools,
                               early_stopping_method='force',
                               max_iterations=10)

In [18]:
query = """Summarize the key points discussed in Nvidia's Q4 2024 earnings call"""
response = chatgpt.invoke(query)
response.content

"As of my last update, Nvidia's Q4 2024 earnings call has not occurred, as my data only goes up to October 2023. Therefore, I cannot provide specific details about the call. However, I can offer guidance on what typically might be discussed in such an earnings call:\n\n1. **Financial Performance**: Discussion of revenue, net income, and earnings per share compared to previous quarters and the same quarter in the previous year.\n\n2. **Segment Performance**: Insights into how different business segments, such as gaming, data center, professional visualization, and automotive, have performed.\n\n3. **Market Trends**: Commentary on market conditions affecting Nvidia's business, such as demand for GPUs, AI advancements, and competition.\n\n4. **Product Updates**: Announcements or updates on new products, technologies, or partnerships.\n\n5. **Guidance**: Forward-looking statements regarding expectations for the next quarter or fiscal year, including revenue projections and strategic priori

In [ ]:
query = """Summarize key points discussed in Databricks Latest Summit"""
response = agent_executor.invoke({"query": query})

Calling web search tool


100%|██████████| 5/5 [00:10<00:00,  2.17s/it]


Error extracting from url: https://www.databricks.com/blog/azure-databricks-guide-data-ai-summit-2023-featuring-akamai-providence-health-services-abn - 'NoneType' object has no attribute 'strip'


In [21]:
response

{'query': 'Summarize key points discussed in Databricks Latest Summit',
 'output': "The Databricks Data + AI Summit 2023 focused on several key themes and announcements:\n\n1. **Generative AI and Lakehouse AI**: Databricks introduced new tools and features to simplify the development of large language model (LLM) applications within the Databricks Lakehouse Platform. Key features include LakehouseIQ, an AI-powered knowledge engine, and Databricks Assistant, an AI-based companion for code and query generation.\n\n2. **Unity Catalog Enhancements**: New features in Unity Catalog were announced to improve data governance and metadata management, including Lakehouse Federation capabilities for accessing and analyzing data across multiple sources, and AI Governance features for managing AI/ML assets.\n\n3. **Data Sharing and Collaboration**: Databricks enhanced data sharing capabilities with the introduction of Databricks Marketplace, Lakehouse Apps, and Delta Sharing, allowing for secure da

In [22]:
from IPython.display import display, Markdown

display(Markdown(response['output']))

The Databricks Data + AI Summit 2023 focused on several key themes and announcements:

1. **Generative AI and Lakehouse AI**: Databricks introduced new tools and features to simplify the development of large language model (LLM) applications within the Databricks Lakehouse Platform. Key features include LakehouseIQ, an AI-powered knowledge engine, and Databricks Assistant, an AI-based companion for code and query generation.

2. **Unity Catalog Enhancements**: New features in Unity Catalog were announced to improve data governance and metadata management, including Lakehouse Federation capabilities for accessing and analyzing data across multiple sources, and AI Governance features for managing AI/ML assets.

3. **Data Sharing and Collaboration**: Databricks enhanced data sharing capabilities with the introduction of Databricks Marketplace, Lakehouse Apps, and Delta Sharing, allowing for secure data sharing and collaboration across platforms.

4. **Data Development Improvements**: New features were introduced to improve performance and usability, such as the English SDK for Apache Spark, Delta Lake 3.0 with a universal format, and Project Lightspeed for enhanced streaming capabilities.

5. **Acquisition of MosaicML**: Databricks announced the acquisition of MosaicML, a platform for training large language models, to enhance its generative AI capabilities.

6. **Governance and AI for Governance**: Databricks emphasized unified governance across data and AI, with new features in Unity Catalog and Lakehouse Monitoring for enhanced observability and governance.

These advancements highlight Databricks' commitment to democratizing AI and analytics, improving data governance, and enhancing collaboration and data sharing capabilities.

In [23]:
query = """Summarize key points discussed in Snowflake Latest Summit"""
response = agent_executor.invoke({"query": query})
display(Markdown(response['output']))

Calling web search tool


100%|██████████| 5/5 [00:06<00:00,  1.38s/it]


Error extracting from url: https://www.databricks.com/blog/azure-databricks-guide-data-ai-summit-2023-featuring-akamai-providence-health-services-abn - 'NoneType' object has no attribute 'strip'


The Databricks Data + AI Summit 2023 focused on several key themes and announcements:

1. **Generative AI and Lakehouse AI**: Databricks introduced new tools and features to simplify the development and deployment of large language models (LLMs) within the Databricks Lakehouse Platform. Key features include LakehouseIQ, an AI-powered knowledge engine, and Databricks Assistant, an AI-based companion for code and query generation.

2. **Unity Catalog Enhancements**: New features in Unity Catalog were announced to improve data governance and metadata management, including Lakehouse Federation capabilities for accessing and analyzing data across multiple sources, and AI Governance features for managing AI/ML assets.

3. **Data Sharing and Collaboration**: Databricks enhanced data sharing capabilities with the introduction of Databricks Marketplace, Lakehouse Apps, and Delta Sharing, allowing for secure data sharing and collaboration across platforms.

4. **Data Development Improvements**: New features were introduced to improve performance and usability, such as the English SDK for Apache Spark, Delta Lake 3.0 with a universal format, and Project Lightspeed for enhanced streaming capabilities.

5. **AI and Machine Learning Innovations**: Databricks announced Lakehouse AI, which includes vector search and model serving capabilities, and MLflow 2.5 for managing LLM operations. The acquisition of MosaicML was also highlighted to enhance generative AI capabilities.

6. **Governance and Monitoring**: New governance features were introduced, including Lakehouse Monitoring for data quality and integrity, and enhanced observability features for better data management.

Overall, the summit emphasized democratizing AI and analytics, improving data governance, and enhancing collaboration and data sharing capabilities within the Databricks ecosystem.

In [24]:
query = """which company's future outlook looks to be better?
        """
response = agent_executor.invoke({"query": query})
display(Markdown(response['output']))

To provide a meaningful answer to your question about which company's future outlook looks better, I would need more specific information. Could you please specify which companies you are interested in comparing?

The agent is doing pretty well but unfortunately it doesn't remember conversations. We will use some user-session based memory to store this and dive deeper into this in the next video.

## Build and Test Multi-User Conversational AI Agent

We will now use `SQLChatMessageHistory` which we learnt in the previous module to store separate conversation histories per user or session.

This will help us build a conversational Agentic Chatbot which will be accessed by many users at the same time

In [ ]:
# removes the memory database file - usually not needed
# you can run this only when you want to remove ALL conversation histories
# ok if you get rm: cannot remove 'memory.db': No such file or directory  because initially no memory exists
# !rm memory.db

rm: cannot remove 'memory.db': No such file or directory


In [26]:
from langchain_community.chat_message_histories import SQLChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# used to retrieve conversation history from database
# based on a specific user or session ID
def get_session_history_db(session_id):
    return SQLChatMessageHistory(session_id, "sqlite:///memory.db")

# create a conversation chain + agent which can load memory based on specific user or session id
agentic_chatbot = RunnableWithMessageHistory(
    agent_executor,
    get_session_history_db,
    input_messages_key="query",
    history_messages_key="history",
)

# function to call the agent show results per user session
from IPython.display import display, Markdown
def chat_with_agent(prompt: str, session_id: str):
    response = agentic_chatbot.invoke({"query": prompt},
                                      {'configurable': { 'session_id': session_id}})
    display(Markdown(response['output']))

Let's now simulate User 1 using the agent

In [28]:
user_id = 'john001'
prompt = "Summarize key points discussed in Databricks Latest Summit"
chat_with_agent(prompt, user_id)

Calling web search tool


 80%|████████  | 4/5 [00:05<00:01,  1.85s/it]

Error extracting from url: https://www.databricks.com/blog/azure-databricks-guide-data-ai-summit-2023-featuring-akamai-providence-health-services-abn - 'NoneType' object has no attribute 'strip'


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
100%|██████████| 5/5 [00:06<00:00,  1.23s/it]


Error extracting from url: https://www.kenwayconsulting.com/blog/databricks-data-ai-world-tour-2023-conference-highlights/ - 'NoneType' object has no attribute 'strip'


The Databricks Data + AI Summit 2023 focused on several key themes and innovations:

1. **Generative AI**: Databricks introduced new tools and features to simplify the development of large language model (LLM) applications within the Databricks Lakehouse Platform. Key features include LakehouseIQ, Databricks Assistant, and Lakehouse AI, which enable building AI models directly on the Data Lakehouse.

2. **Unity Catalog Enhancements**: New features were announced to simplify data management and governance, including Lakehouse Federation Capabilities, AI Governance, and Lakehouse Monitoring and Observability.

3. **Data Sharing and Collaboration**: Databricks enhanced its data sharing capabilities with the introduction of Databricks Marketplace, Lakehouse Apps, and Delta Sharing, allowing for efficient data collaboration and sharing.

4. **Data Development Improvements**: New features like the English SDK for Apache Spark, Delta Lake 3.0, and Project Lightspeed were introduced to improve performance, compatibility, and scalability in data development.

5. **Governance and AI**: Databricks emphasized unified governance across data and AI, with new features in Unity Catalog and Lakehouse Federation to manage data and AI assets effectively.

6. **Lakehouse Collaboration**: Delta Sharing and Databricks Marketplace were highlighted as key features for cross-platform collaboration and data sharing.

Overall, the summit showcased Databricks' commitment to innovation in AI and data management, aiming to democratize AI and enhance data-driven decision-making across organizations.

In [29]:
prompt = "What about Intel?"
chat_with_agent(prompt, user_id)

Calling web search tool


100%|██████████| 5/5 [00:06<00:00,  1.38s/it]

Error extracting from url: https://www.databricks.com/blog/whats-new-unity-catalog-data-and-ai-summit-2023 - 'NoneType' object has no attribute 'strip'


At the Databricks Data + AI Summit 2023, Intel's involvement was highlighted through their collaboration with Databricks to enhance AI and data processing capabilities. Intel's focus was on optimizing performance and efficiency for AI workloads, leveraging their hardware advancements to support Databricks' Lakehouse Platform. This partnership aims to accelerate data processing and AI model training, providing users with improved computational power and efficiency. The collaboration underscores Intel's commitment to advancing AI technologies and supporting data-driven innovations in partnership with leading data platforms like Databricks.

In [30]:
prompt = "Which company seems to be doing better?"
chat_with_agent(prompt, user_id)

To determine which company, between Databricks and Intel, is doing better, we would need to consider various factors such as financial performance, market position, innovation, partnerships, and overall impact in their respective industries. 

1. **Databricks**: 
   - **Innovation and Growth**: Databricks has been at the forefront of data and AI innovation, particularly with its Lakehouse Platform, which combines data warehousing and AI capabilities. The company's focus on generative AI and data governance has positioned it as a leader in the data analytics space.
   - **Market Position**: Databricks has seen significant growth and investment, with a strong presence in the cloud data platform market. Its partnerships and integrations with major cloud providers enhance its market reach.

2. **Intel**: 
   - **Technological Advancements**: Intel continues to be a leader in semiconductor manufacturing and computing technology. Its focus on AI and data processing capabilities, as seen in collaborations with companies like Databricks, highlights its commitment to staying relevant in the AI and data processing sectors.
   - **Market Challenges**: Intel faces competition from other semiconductor companies like AMD and NVIDIA, which have also made significant strides in AI and data processing technologies.

To provide a more precise assessment, I would need to look at recent financial reports, market analyses, and industry trends. If you would like, I can search for the latest information on both companies to provide a more detailed comparison. Would you like me to do that?

Let's now simulate User 2 using the agent

In [31]:
user_id = 'bond007'
prompt = "how is the weather in Bangalore today? Show detailed statistics"
chat_with_agent(prompt, user_id)

Calling weather tool
Calling web search tool


 60%|██████    | 3/5 [00:02<00:01,  1.14it/s]

Error extracting from url: https://www.freepressjournal.in/india/bengaluru-weather-update-for-july-16 - 'NoneType' object has no attribute 'strip'


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
 80%|████████  | 4/5 [00:03<00:00,  1.22it/s]

Error extracting from url: https://www.easeweather.com/asia/india/karnataka/bangalore-urban/bengaluru/july - 'NoneType' object has no attribute 'strip'


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
100%|██████████| 5/5 [00:04<00:00,  1.12it/s]


Error extracting from url: https://www.weather25.com/asia/india/karnataka/bangalore?page=month&month=July - 'NoneType' object has no attribute 'strip'


The weather in Bangalore today is as follows:

- **Temperature**: The minimum temperature is around 21°C, and the maximum temperature is expected to reach 28°C.
- **Weather Condition**: Rain is expected throughout the day.
- **Humidity**: The humidity level is at 82%.
- **Pressure**: Atmospheric pressure is at 1009 hPa.
- **Wind**: The wind speed is around 6.48 km/h, coming from 257 degrees with gusts up to 10.33 km/h.
- **Sunrise and Sunset**: The sun rises at 06:02 AM and sets at 06:50 PM.

The air quality index in Bangalore is satisfactory, with PM2.5 at 67 and PM10 at 38.

This information provides a detailed overview of the current weather conditions in Bangalore.

In [33]:
user_id = 'bond007'
prompt = "what about Dubai?"
chat_with_agent(prompt, user_id)

Calling weather tool


I couldn't retrieve the weather data for Dubai at the moment. There might be an issue with accessing the weather information for that location. You might want to try checking a reliable weather website or app for the latest updates.

In [24]:
user_id = 'bond007'
prompt = "which city is hotter?"
chat_with_agent(prompt, user_id)

Dubai is currently hotter than Bangalore. The temperature in Dubai is 16.2°C (61.2°F), while in Bangalore, it is 18.3°C (64.9°F).